In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import neweggutils as nu
import re

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df = pd.read_csv('final.csv')

In [4]:
df.drop(labels=['Unnamed: 0'], inplace=True, axis=1)

In [6]:
df_clean = df.copy()

# Drop duplicates and unnecessary columns

In [9]:
df_clean = df_clean[~df_clean.duplicated()]

In [12]:
df_clean.columns.values

array(['price', 'link', 'Brand', 'Series', 'Model', 'Part Number', 'Type',
       'Form Factor', 'Usage', 'Colors', 'Processor',
       'Processor Main Features', 'Cache Per Processor', 'Memory',
       'Storage', 'Optical Drive', 'Graphics', 'Power Supply',
       'Operating System', 'CPU Type', 'CPU Speed', 'L3 Cache Per CPU',
       'CPU Main Features', 'GPU/VGA Type', 'Video Memory',
       'Virtual Reality Ready', 'Memory Capacity', 'Memory Speed',
       'Memory Spec', 'Memory Slot (Total)', 'Maximum Memory Supported',
       'Optane Memory', 'HDD', 'HDD RPM', 'Optical Drive Type',
       'Screen Size', 'LAN Speed', 'WLAN', 'Bluetooth', 'Mouse Type',
       'Keyboard Type', 'Dimensions (H x W x D)', 'Weight',
       'Graphics Interface', 'SSD', 'Keyboard', 'Storage Spec',
       'Optical Drive Spec', 'Media Slots', 'Display Feature',
       'Front USB', 'Mouse', 'Special Features', 'Package Contents',
       'Software Included', 'Audio Chipset', 'LAN Chipset', 'Video Ports',
    

In [15]:
df_clean = df_clean.drop(labels=['Series', 'Model', 'Part Number', 'Model', 'Colors', 'Cache Per Processor',
                                 'Optical Drive', 'L3 Cache Per CPU', 'Virtual Reality Ready', 'Optane Memory',
                                 'LAN Speed', 'WLAN', 'Bluetooth', 'Mouse Type', 'Keyboard Type',
                                 'Dimensions (H x W x D)', 'Keyboard', 'Mouse', 'Special Features',
                                 'Package Contents', 'Software Included', 'Audio Chipset', 'LAN Chipset',
                                 'Speaker Configuration', 'Front Audio Ports', 'L2 Cache Per CPU', 
                                 'COM', 'Speaker Features', 'FIR', 'Card Reader', 'Installed Qty', 'Touchscreen',
                                 'LPT', 'External Bays', 'Internal Bays', 'Package Type', 'Optical Drive 2',
                                 'Optical Drive 2 Type', 'Front IEEE 1394'], axis=1)

In [17]:
df_clean = df_clean.drop(labels=['Power Supply', 'Memory Speed', 'Memory Spec', 'Memory Slot (Total)',
                                 'Maximum Memory Supported', 'HDD RPM', 'Weight', 'Media Slots', 'Display Feature',
                                 'HDD Interface', 'Ethernet', 'Chipset', 'Audio Features', 'Memory Slot (Available)',
                                 'Graphics Card', 'Screen Type', 'PCI Slots (Available/Total)', 'Monitor',
                                 'Audio', 'Speaker', 'SLI / Crossfire', 'Max Supported Qty'], axis=1)

In [26]:
df_clean.to_csv('final_pared.csv')

In [42]:
df_clean.reset_index(inplace=True, drop=True)

## Grab processor brand, cores, and speed

In [44]:
df_clean['proc_brand'] = df_clean['Processor'].str.extract(r'^(INTEL|AMD)', flags=re.IGNORECASE)

In [74]:
df_clean.drop(labels=['Type', 'Usage'], axis=1, inplace=True)

In [88]:
df_clean['Processor Main Features'].str.extract(r'(-Core|Core)', flags=re.IGNORECASE)

0    0.252858
dtype: float64

In [141]:
df_clean['proc_cores'] = df_clean['Processor'].apply(lambda x: nu.num_cores(str(x)))

In [211]:
pat = re.compile('([0-9]\.[0-9]?)')
df_clean['proc_speed'] = df_clean['CPU Speed'].str.extract(pat)

In [159]:
df = df_clean.copy()

In [161]:
df.to_csv('grab_procs.csv')

## Grab RAM capacity and type (ddr2 vs ddr3 vs ddr4)

In [219]:
df_clean['mem_type'] = df_clean['Memory Capacity'].apply(lambda x: nu.ram_type(str(x)))

In [220]:
df_clean['mem_type'].notna().sum()

2500

In [221]:
df_clean['mem_cap'] = df_clean['Memory Capacity'].apply(lambda x: nu.ram_cap(str(x)))

In [222]:
df_clean.to_csv('final_dirty1.csv')

## Get storage capacity and type

In [247]:
df['Storage'].sample(50)

1553                     2 TB
2188                   750 GB
1513               250 GB HDD
376                       NaN
2881                     2 TB
2479          250 GB 7200 rpm
498                       NaN
2420     250 GB SATA 7200 RPM
188                       NaN
360                       NaN
98                        NaN
2023                      NaN
500                       NaN
1761                     1 TB
511                360 GB SSD
868                    500 GB
581                       NaN
1979                 1 TB HDD
1351                      NaN
763                       NaN
2171       2 TB SATA 7200 RPM
1856     250 GB SATA 7200 RPM
2894       1 TB SATA 7200 RPM
2936       1 TB SATA 7200 RPM
2223                      NaN
1895       1 TB SATA 7200 RPM
1739                     2 TB
522                  2 TB HDD
2068       1 TB SATA 7200 RPM
1932                      NaN
793             2 TB 7200 RPM
329     3 TB HDD + 120 GB SSD
2231                      NaN
2377      

In [242]:
df['SSD'].sample(50)

2306       NaN
215     128 GB
1230       NaN
1096    240 GB
1453        No
1025       NaN
2639        No
945        NaN
1516        No
994         No
1771       NaN
898        NaN
1558        No
2473       NaN
167         No
2091        No
2717        No
2917       NaN
1752       NaN
857        NaN
859        NaN
1978        No
351     120 GB
48      512 GB
2392        No
2032        No
2943    128 GB
2625        No
1724        No
2178        No
1214       NaN
765     120 GB
2754        No
922        NaN
2511        No
1228       NaN
1319        No
2324       NaN
1537        No
1312        No
1667        No
1631       NaN
487        NaN
2346       NaN
2270       NaN
910        NaN
2769       NaN
140        NaN
2017        No
1083    240 GB
Name: SSD, dtype: object